In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [30]:
import pandas as pd

# Read CSV and preprocess the data
future = pd.read_csv("./data/test_future.csv")
future = future.sort_values(["종목코드", "일자"]).reset_index(drop=True)
future = future.drop(["거래량", "시가", "고가", "저가"], axis=1)

def calculate_return(df):
    return df.groupby('종목코드')['종가'].pct_change()

def calculate_std(df):
    return df.groupby('종목코드')['수익률'].std()

def calculate_final_return(df):
    first_day_closing = df.groupby('종목코드')['종가'].first()
    last_day_closing = df.groupby('종목코드')['종가'].last()
    return ((last_day_closing - first_day_closing) / first_day_closing).to_frame()

# Calculate returns and add to the DataFrame
future['수익률'] = calculate_return(future)

# Calculate standard deviation
std_df = calculate_std(future)
std_df = std_df.reset_index()

# Calculate final return and Sharpe ratio
final_return_df = calculate_final_return(future)
result = std_df.merge(final_return_df, on='종목코드')
result = result.rename(columns={"수익률" : "std", "종가" : "final_return"})
result

,종목코드,std,final_return
0,A000020,0.015429,0.119792
1,A000040,0.009978,-0.104478
2,A000050,0.006591,-0.035680
3,A000070,0.012383,0.056657
4,A000080,0.010666,-0.050218
...,...,...,...
1992,A375500,0.013871,0.012397
1993,A378850,0.021294,0.026316
1994,A383220,0.017311,0.010046
1995,A383310,0.023728,0.030547


In [31]:
# '종목코드'와 '일자'를 기준으로 DataFrame을 정렬
df1 = future.sort_values(['종목코드', '일자'])

# '종목코드', '일자_그룹', 그리고 연속된 15일의 일자에 대해 pivot 수행
df_pivot = df1.pivot_table(index=['종목코드'], columns=df1.groupby(['종목코드']).cumcount() + 1, values='수익률')

# 컬럼 이름을 'return1', 'return2', ..., 'return15'으로 변경
df_pivot.columns = [f'return{i}' for i in range(1, 15)]

# Reset index
df_pivot = df_pivot.reset_index()

# df2와 merge
result = result.merge(df_pivot, on='종목코드', how='left')
result

,종목코드,std,final_return,return1,return2,return3,return4,return5,return6,return7,return8,return9,return10,return11,return12,return13,return14
0,A000020,0.015429,0.119792,0.050000,-0.000992,0.003972,-0.009891,0.023976,-0.001951,0.012708,0.018340,-0.009479,0.009569,0.007583,0.010348,0.001862,-0.000929
1,A000040,0.009978,-0.104478,0.005970,-0.016320,0.006033,-0.005997,-0.004525,0.001515,-0.016641,-0.013846,-0.021841,-0.023923,-0.006536,0.004934,-0.009820,-0.008264
2,A000050,0.006591,-0.035680,-0.000964,-0.004826,0.001940,0.000968,0.000000,0.002901,-0.003857,-0.003872,0.000972,-0.010680,0.002944,-0.004892,-0.020649,0.004016
3,A000070,0.012383,0.056657,0.026912,0.015172,-0.004076,0.024557,0.003995,0.011936,-0.007864,-0.007926,0.007989,-0.010568,0.010681,-0.005284,-0.005312,-0.004005
4,A000080,0.010666,-0.050218,-0.017467,0.006667,0.008830,-0.006565,-0.013216,0.013393,0.000000,-0.013216,-0.002232,-0.008949,0.000000,0.011287,-0.011161,-0.018059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,A375500,0.013871,0.012397,-0.004132,0.016598,0.006803,0.005405,0.002688,-0.002681,0.001344,-0.006711,-0.004054,-0.013569,0.039890,-0.009259,-0.013351,-0.005413
1993,A378850,0.021294,0.026316,0.040046,0.025303,-0.010730,-0.034707,0.002247,0.011211,0.021064,0.001086,0.008677,-0.030108,0.007761,0.016502,-0.020563,-0.008840
1994,A383220,0.017311,0.010046,-0.010819,0.037500,-0.003765,0.000000,-0.012850,0.005360,-0.004570,-0.000765,0.002297,-0.002292,0.035222,-0.004438,0.000743,-0.029696
1995,A383310,0.023728,0.030547,-0.001608,0.020934,0.041009,0.036364,0.007310,0.004354,0.001445,0.005772,-0.053085,0.013636,-0.001495,0.000000,-0.022455,-0.018377


In [ ]:
import pandas as pd
import numpy as np

# '종목코드'와 '일자'를 기준으로 DataFrame을 정렬
df1 = future.sort_values(['종목코드', '일자'])

# 종목별로 수익률 컬럼에 대해 그룹화하고, 각 그룹에 대해 15일의 수익률을 가져옵니다.
df1['일자_그룹'] = df1.groupby('종목코드').cumcount() // 15

# '종목코드', '일자_그룹', 그리고 연속된 15일의 일자에 대해 pivot 수행
df_pivot = df1.pivot_table(index=['종목코드', '일자_그룹'], columns=df1.groupby(['종목코드', '일자_그룹']).cumcount() + 1, values='수익률')

# 컬럼 이름을 'return1', 'return2', ..., 'return15'으로 변경
df_pivot.columns = [f'return{i}' for i in range(1, 16)]

# Reset index
df_pivot = df_pivot.reset_index()

# df2와 merge
df2 = df2.merge(df_pivot, on='종목코드', how='left')

## 투자종목 선정 (상위 200 , 하위 200)

In [106]:
xgboost = pd.read_csv("./sub/XGBoost_-0.8.csv")

xgboost = xgboost.sort_values("순위", ascending=False)

xgboost_short = xgboost[:200]
xgboost_long = xgboost[-200:]

xgboost_short_list = xgboost_short['종목코드'].to_list()
xgboost_long_list = xgboost_long['종목코드'].to_list()

In [107]:
invest = result.sort_values('final_return')
short = invest[invest['종목코드'].isin(xgboost_short_list)]
long = invest[invest['종목코드'].isin(xgboost_long_list)]

# 총자산 최종수익률
asset_return = (short['final_return'].sum() * -1 + long['final_return'].sum()) / 400

# 연율화된 총자산 최종 수익률
annualised_return = asset_return * 250 / 15
annualised_return

print("총자산 최종수익률", asset_return.round(5) * 100, "\n연율화된 총자산 최종 수익률" ,annualised_return.round(5) * 100)

총자산 최종수익률 -1.476 
연율화된 총자산 최종 수익률 -24.593


In [109]:
allPortfolio = short.merge(long, how='outer')
allPortfolio

df3 = allPortfolio[['return1', 'return2', 'return3', 'return4', 'return5', 'return6', 'return7', 'return8', 'return9', 'return10', 'return11', 'return12', 'return13', 'return14']] * 250
df3

,return1,return2,return3,return4,return5,return6,return7,return8,return9,return10,return11,return12,return13,return14
0,-14.845606,-1.388889,-7.110208,-31.756404,1.796407,5.499405,-5.526469,-8.328376,-3.076923,-3.894081,8.544304,-5.507956,-11.576971,-2.132546
1,-2.035278,-0.341997,1.027397,-9.208731,-5.311615,-0.723589,-14.150943,-5.769231,-7.086614,1.620746,0.000000,4.025765,-6.339144,0.000000
2,5.331089,-5.494505,-3.089888,-1.706485,-4.295533,0.000000,1.748252,-9.837963,-6.927711,-1.239157,0.933998,-5.272953,-7.921420,-5.235602
3,30.723906,2.623688,-4.821958,-17.019667,6.493506,-2.768987,-6.400000,-3.284072,-7.487521,-17.581475,15.682657,9.548611,-42.642140,5.040323
4,-6.449165,1.557632,-5.804954,0.990491,-8.879242,1.636661,-3.455285,-4.328112,-9.437919,-1.743679,12.291484,-5.230126,-10.042735,0.222618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,5.700326,74.840764,-11.029412,0.854701,5.536627,12.500000,0.793651,13.053797,-11.278195,-11.417323,2.062706,6.955810,-0.398089,18.740032
396,5.898681,43.050847,-4.193175,75.000000,1.696833,0.000000,10.112360,-5.939525,6.637168,-12.392241,5.102041,8.333333,12.903226,-11.758691
397,4.419192,43.424318,-10.042283,11.563877,26.315789,16.666667,17.857143,-10.416667,-19.130435,0.941620,15.009381,3.097345,43.706294,-4.836310
398,23.736128,2.252252,-3.627232,12.174405,74.514039,-4.159734,5.922166,42.148760,-28.642150,-6.389776,0.409836,2.045827,20.698052,23.988006


## 샤프 계산

In [111]:
# n일 째 수익률의 평균 계산
mean_return = df3.mean(axis=0)
mean_return

return1     0.765236
return2     1.845779
return3     1.803565
return4     0.351109
return5     0.038003
return6     2.217882
return7    -0.166864
return8     1.695570
return9    -3.372134
return10    0.116476
return11    2.730098
return12    1.304434
return13   -1.053545
return14   -1.813950
dtype: float64

In [113]:
# 전체 수익률의 평균 계산
total_mean = df3.mean(axis=1).mean(axis=0)
total_mean

0.46154689746948285

In [116]:
# N번째 매매일의 일간 수익률 평균 수익률과 일간 수익률의 평균의 차이의 제곱의 합을 계산
diff_sum = (mean_return - total_mean) ** 2
diff_sum

return1      0.092227
return2      1.916098
return3      1.801012
return4      0.012197
return5      0.179390
return6      3.084714
return7      0.394901
return8      1.522813
return9     14.697112
return10     0.119074
return11     5.146322
return12     0.710459
return13     2.295504
return14     5.177884
dtype: float64

In [115]:
# 위 결과를 n - 2로 나누고, 그 제곱근을 구함
allPortfolio_std = np.sqrt(diff_sum / (15 - 2))
allPortfolio_std = allPortfolio_std.to_frame().sum()[0]
allPortfolio_std

5.065375130638352

In [124]:
print("실제 샤프 :", (annualised_return.round(5) * 100 - (3.5)) / allPortfolio_std)

실제 샤프 : -5.5460847963810425


In [ ]:
#RMSE는 예측값과 실제 값 간의 차이를 제곱하여 평균을 낸 후, 그 결과의 제곱근을 구하는 방식으로 계산됩니다. RMSE 값이 작을수록 예측이 정확하다는 것을 의미합니다.
#평균 절대 오차(Mean Absolute Error, MAE) 또한 유사성을 판단하는 데 사용할 수 있습니다. MAE는 예측값과 실제 값의 차이의 절대값을 평균한 값입니다.
#결정 계수(R-squared, R^2) 는 예측값이 실제 값에 얼마나 가까운지를 나타내는 통계적 척도입니다. R^2 값이 1에 가까울수록 예측이 실제 값을 잘 설명하고 있다는 것을 의미합니다.

# 실제 종가 데이터
file_dir = "C:\Users\82107\OneDrive - 경희대학교\빅데이터응용학과\KRX 박재균/KRX 평가용.csv" # 여기에 csv 경로 입력
df_actual = pd.read_csv(file_dir)
df_actual['일자'] = pd.to_datetime(df_actual['일자'].astype(str))
df_actual_20230621 = df_actual[df_actual['일자'] == '2023-06-21']

# 예측 종가 데이터
df_predicted = pd.read_csv('/mnt/data/predicted_prices.csv')
df_predicted['일자'] = pd.to_datetime(df_predicted['일자'].astype(str))
df_predicted_20230621 = df_predicted[df_predicted['일자'] == '2023-06-21']

# RMSE 계산
rmse = np.sqrt(mean_squared_error(df_actual_20230621['종가'], df_predicted_20230621['종가']))

# MAE 계산
mae = mean_absolute_error(df_actual_20230621['종가'], df_predicted_20230621['종가'])

# R^2 계산
r2 = r2_score(df_actual_20230621['종가'], df_predicted_20230621['종가'])


print(rmse, mae, r2)